# Notebook \#3 - Operator Learning and Optimal Control 
version: LNO

for some instructions regarding use of special functions from this repo, consider starting from exploring `playground_deeponet.ipynb`

In [1]:
import torch
from utils.scripts import solve_optimization
from models.lno import LNO1d, LNO1d_extended

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
m = 200
modes = 8
width = 4
hidden_layer = 64


model = LNO1d(modes=modes, width=width, activation="silu", hidden_layer = hidden_layer).to(device)

ckpt = torch.load("trained_models/linear/lno/unsupervised/epoch[670]_model_time_[20250718_232320]_loss_[0.0011].pth", map_location=device)
model.load_state_dict(ckpt["model_state_dict"])

w = [10, 1, 1, 0, 0]
initial_guess = torch.rand((1, m), dtype=torch.float32, device=device, requires_grad=True)

analytics, x, u, x_optimal, u_optimal = solve_optimization(
    model, 'linear',
    initial_guess,
    lr=0.001,
    architecture="lno", 
    w=w,
    num_epochs=5000,
    m=m,
    device=device,
    logging=True
)

Epoch   50 | Loss: 0.684169 | rel_err_u: 3.5953, rel_err_x: 0.4820
Epoch  100 | Loss: 0.604572 | rel_err_u: 3.4798, rel_err_x: 0.4703
Epoch  150 | Loss: 0.545865 | rel_err_u: 3.3590, rel_err_x: 0.4560
Plot saved to found_trajectories/linear/lno/plot.png
Epoch  200 | Loss: 0.500890 | rel_err_u: 3.2362, rel_err_x: 0.4403
Epoch  250 | Loss: 0.465107 | rel_err_u: 3.1126, rel_err_x: 0.4235
Epoch  300 | Loss: 0.435577 | rel_err_u: 2.9890, rel_err_x: 0.4062
Epoch  350 | Loss: 0.410481 | rel_err_u: 2.8660, rel_err_x: 0.3884
Plot saved to found_trajectories/linear/lno/plot.png
Epoch  400 | Loss: 0.388684 | rel_err_u: 2.7446, rel_err_x: 0.3706
Epoch  450 | Loss: 0.369450 | rel_err_u: 2.6252, rel_err_x: 0.3529
Epoch  500 | Loss: 0.352292 | rel_err_u: 2.5085, rel_err_x: 0.3354
Epoch  550 | Loss: 0.336863 | rel_err_u: 2.3948, rel_err_x: 0.3183
Plot saved to found_trajectories/linear/lno/plot.png
Epoch  600 | Loss: 0.322923 | rel_err_u: 2.2842, rel_err_x: 0.3015
Epoch  650 | Loss: 0.310287 | rel_err

In [3]:
print(f"Final objective loss: {analytics['obj'][-1]}")

Final objective loss: 0.1925065517425537


<image src="found_trajectories/linear/lno/plot.png" width=600>

In [8]:
m = 200
modes = 8
width = 4
hidden_layer = 64

model = LNO1d(modes=modes, width=width, activation="silu", hidden_layer = hidden_layer).to(device)
ckpt = torch.load("trained_models/oscillatory/lno/unsupervised/epoch[1000]_model_time_[20250718_232530]_loss_[0.0017].pth", map_location=device)
model.load_state_dict(ckpt["model_state_dict"])

w = [10, 1, 1, 0, 0.1]
initial_guess = torch.rand((1, m), dtype=torch.float32, device=device, requires_grad=True)

analytics, x, u, x_optimal, u_optimal = solve_optimization(
    model, 'oscillatory',
    initial_guess,
    lr=0.001,
    architecture="lno", 
    w=w,
    num_epochs = 10000,
    m=m,
    device=device,
    result=True
)

Epoch   50 | Loss: 0.319972 | rel_err_u: 114.3646, rel_err_x: 4.5346
Epoch  100 | Loss: 0.257602 | rel_err_u: 109.1187, rel_err_x: 4.3778
Epoch  150 | Loss: 0.212674 | rel_err_u: 104.0166, rel_err_x: 4.1941
Plot saved to found_trajectories/oscillatory/lno/plot.png
Epoch  200 | Loss: 0.179017 | rel_err_u: 98.9773, rel_err_x: 3.9906
Epoch  250 | Loss: 0.152756 | rel_err_u: 93.9913, rel_err_x: 3.7746
Epoch  300 | Loss: 0.131582 | rel_err_u: 89.0935, rel_err_x: 3.5532
Epoch  350 | Loss: 0.114083 | rel_err_u: 84.3235, rel_err_x: 3.3317
Plot saved to found_trajectories/oscillatory/lno/plot.png
Epoch  400 | Loss: 0.099366 | rel_err_u: 79.7111, rel_err_x: 3.1140
Epoch  450 | Loss: 0.086814 | rel_err_u: 75.2749, rel_err_x: 2.9028
Epoch  500 | Loss: 0.076010 | rel_err_u: 71.0257, rel_err_x: 2.6997
Epoch  550 | Loss: 0.066648 | rel_err_u: 66.9660, rel_err_x: 2.5058
Plot saved to found_trajectories/oscillatory/lno/plot.png
Epoch  600 | Loss: 0.058495 | rel_err_u: 63.0946, rel_err_x: 2.3218
Epoch  

In [5]:
print(f"Final objective loss: {analytics['obj'][-1]}")

Final objective loss: 0.0015614270232617855


<image src="found_trajectories/oscillatory/lno/plot.png" width=600>

In [5]:
m = 200
modes = 8
width = 4
hidden_layer = 64

model = LNO1d(width, modes, hidden_layer=hidden_layer).to(device)

ckpt = torch.load("trained_models/polynomial_tracking/lno/unsupervised/epoch[1000]_model_time_[20250718_233449]_loss_[0.0127].pth", map_location=device)
model.load_state_dict(ckpt["model_state_dict"])

w = [100, 1, 1, 1, 0]
initial_guess = torch.rand((1, m), dtype=torch.float32, device=device, requires_grad=True)

analytics, x, u, x_optimal, u_optimal = solve_optimization(
    model, 'polynomial_tracking',
    initial_guess,
    lr=0.001,
    architecture="lno", 
    w=w,
    num_epochs = 10000,
    m=m,
    device=device,
    result=True
)

Epoch   50 | Loss: 1.717599 | rel_err_u: 2.1812, rel_err_x: 1.3872
Epoch  100 | Loss: 1.229454 | rel_err_u: 2.0968, rel_err_x: 1.3794
Epoch  150 | Loss: 0.910929 | rel_err_u: 2.0268, rel_err_x: 1.3696
Plot saved to found_trajectories/polynomial_tracking/lno/plot.png
Epoch  200 | Loss: 0.702320 | rel_err_u: 1.9670, rel_err_x: 1.3561
Epoch  250 | Loss: 0.565567 | rel_err_u: 1.9135, rel_err_x: 1.3387
Epoch  300 | Loss: 0.475413 | rel_err_u: 1.8637, rel_err_x: 1.3175
Epoch  350 | Loss: 0.415122 | rel_err_u: 1.8159, rel_err_x: 1.2930
Plot saved to found_trajectories/polynomial_tracking/lno/plot.png
Epoch  400 | Loss: 0.373730 | rel_err_u: 1.7691, rel_err_x: 1.2659
Epoch  450 | Loss: 0.344285 | rel_err_u: 1.7230, rel_err_x: 1.2365
Epoch  500 | Loss: 0.322442 | rel_err_u: 1.6774, rel_err_x: 1.2055
Epoch  550 | Loss: 0.305485 | rel_err_u: 1.6323, rel_err_x: 1.1733
Plot saved to found_trajectories/polynomial_tracking/lno/plot.png
Epoch  600 | Loss: 0.291741 | rel_err_u: 1.5877, rel_err_x: 1.140

KeyboardInterrupt: 

In [7]:
print(f"Final objective loss: {analytics['obj'][-1]}")

Final objective loss: 0.14918164908885956


<image src="found_trajectories/polynomial_tracking/lno/plot.png" width=600>

In [9]:
m = 200

modes = [8, 8]
width = 4
hidden_layer = 64
model = LNO1d_extended(modes=modes, width=width, activation="tanh", depth=2, active_last=True, hidden_layer=hidden_layer).to(device)

ckpt = torch.load("trained_models/nonlinear/lno/unsupervised/epoch[360]_model_time_[20250719_151537]_loss_[0.0597].pth", map_location=device)
model.load_state_dict(ckpt["model_state_dict"])

w = [100, 1, 1, 1, 0]
initial_guess = torch.rand((1, m), dtype=torch.float32, device=device, requires_grad=True)

analytics, x, u, x_optimal, u_optimal = solve_optimization(
    model, 'nonlinear',
    initial_guess,
    lr=0.001,
    architecture="lno", 
    w=w,
    num_epochs = 30000,
    m=m,
    bounds = [-1.5, 1.5],
    device=device,
    result=True,
    early_stopping=True,
)

Epoch   50 | Loss: 10.389113 | rel_err_u: 1.7321, rel_err_x: 0.3173
Epoch  100 | Loss: 7.086969 | rel_err_u: 1.6767, rel_err_x: 0.2947
Epoch  150 | Loss: 5.018788 | rel_err_u: 1.6399, rel_err_x: 0.2807
Plot saved to found_trajectories/nonlinear/lno/plot.png
Epoch  200 | Loss: 3.691982 | rel_err_u: 1.6170, rel_err_x: 0.2721
Epoch  250 | Loss: 2.832678 | rel_err_u: 1.6026, rel_err_x: 0.2667
Epoch  300 | Loss: 2.273173 | rel_err_u: 1.5931, rel_err_x: 0.2630
Epoch  350 | Loss: 1.904635 | rel_err_u: 1.5866, rel_err_x: 0.2604
Plot saved to found_trajectories/nonlinear/lno/plot.png
Epoch  400 | Loss: 1.658357 | rel_err_u: 1.5814, rel_err_x: 0.2583
Epoch  450 | Loss: 1.491769 | rel_err_u: 1.5767, rel_err_x: 0.2564
Epoch  500 | Loss: 1.377275 | rel_err_u: 1.5718, rel_err_x: 0.2544
Epoch  550 | Loss: 1.296346 | rel_err_u: 1.5668, rel_err_x: 0.2525
Plot saved to found_trajectories/nonlinear/lno/plot.png
Epoch  600 | Loss: 1.237515 | rel_err_u: 1.5618, rel_err_x: 0.2507
Epoch  650 | Loss: 1.192442

In [10]:
print(f"Final objective loss: {analytics['obj'][-1]}")

Final objective loss: -0.03414168953895569


<image src="found_trajectories/nonlinear/lno/plot.png" width=600>

In [14]:
m = 200

modes = 32
width = 16
hidden_layer = 128

model = LNO1d(width, modes, activation = "silu",batch_norm=True, active_last=True, hidden_layer=hidden_layer).to(device)

ckpt = torch.load("trained_models/singular_arc/lno/attempt_started20250720_215514/epoch[1000]_model_time_[20250720_215514]_loss_[0.0148].pth", map_location=device)
model.load_state_dict(ckpt["model_state_dict"])

w = [50, 1, 1, 0, 10]
initial_guess = torch.rand((1, m), dtype=torch.float32, device=device, requires_grad=True)-3 

analytics, x, u, x_optimal, u_optimal = solve_optimization(
    model, 'singular_arc',
    initial_guess,
    lr=0.001,
    architecture="lno", 
    w=w,
    num_epochs = 100000,
    early_stopping=True,
    m=m,
    bounds = [-3.5, 0],
    device=device,
    result=True,
    plots=False,
)

Epoch   50 | Loss: 10.844107 | rel_err_u: 1.0218, rel_err_x: 1.2115
Epoch  100 | Loss: 10.369057 | rel_err_u: 0.9943, rel_err_x: 1.1598
Epoch  150 | Loss: 9.939803 | rel_err_u: 0.9668, rel_err_x: 1.1077
Plot saved to found_trajectories/singular_arc/lno/plot.png
Epoch  200 | Loss: 9.548416 | rel_err_u: 0.9397, rel_err_x: 1.0556
Epoch  250 | Loss: 9.189808 | rel_err_u: 0.9132, rel_err_x: 1.0041
Epoch  300 | Loss: 8.859728 | rel_err_u: 0.8873, rel_err_x: 0.9536
Epoch  350 | Loss: 8.555278 | rel_err_u: 0.8620, rel_err_x: 0.9046
Plot saved to found_trajectories/singular_arc/lno/plot.png
Epoch  400 | Loss: 8.274195 | rel_err_u: 0.8374, rel_err_x: 0.8572
Epoch  450 | Loss: 8.015017 | rel_err_u: 0.8135, rel_err_x: 0.8120
Epoch  500 | Loss: 7.776542 | rel_err_u: 0.7902, rel_err_x: 0.7690
Epoch  550 | Loss: 7.557256 | rel_err_u: 0.7676, rel_err_x: 0.7285
Plot saved to found_trajectories/singular_arc/lno/plot.png
Epoch  600 | Loss: 7.355408 | rel_err_u: 0.7455, rel_err_x: 0.6906
Epoch  650 | Loss

KeyboardInterrupt: 

In [11]:
print(f"Final objective loss: {analytics['obj'][-1]}")

Final objective loss: 2.183225393295288


<image src="found_trajectories/singular_arc/lno/plot.png" width=600>